In [1]:
import pandas as pd

df = pd.read_csv("df_features.csv")
df.head()


,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,...,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUIWN(kBtu/sf),SteamUse(kBtu),Agedubatiment,UsageCount,ParkingRatio,PrimaryUseRatio
0,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,...,Unknown,12371.421298,Unknown,60.0,84.300003,2003882.00,98,1,0.000000,1.000000
1,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,...,Restaurant,4622.000000,Unknown,61.0,97.900002,0.00,29,3,0.145453,0.809918
2,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,...,Unknown,12371.421298,Unknown,43.0,97.699997,21566554.00,56,1,0.205748,0.791220
3,NonResidential,Hotel,98101.0,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,...,Unknown,12371.421298,Unknown,56.0,113.300003,2214446.25,99,1,0.000000,1.000000
4,NonResidential,Hotel,98121.0,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,...,Swimming Pool,0.000000,Unknown,75.0,118.699997,0.00,45,3,0.353115,0.703070


In [2]:
TARGET = "SiteEUIWN(kBtu/sf)"

X = df.drop(columns=[TARGET])
y = df[TARGET]

print("X :", X.shape)
print("y :", y.shape)


X : (1521, 27)
y : (1521,)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print("X_train :", X_train.shape)
print("X_test  :", X_test.shape)


X_train : (1216, 27)
X_test  : (305, 27)
